In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
from accelerate import infer_auto_device_map
import random
from PIL import Image

# Carica il modello dalla piattaforma Hugging Face
model_id = "stabilityai/stable-diffusion-xl-base-1.0"

# Configura la pipeline
pipeline = StableDiffusionXLPipeline.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    use_safetensors=True  # Per performance migliori
).to("cuda")

In [ ]:
# Prompt di esempio
prompt = "A futuristic cityscape illuminated by neon lights at sunset, with towering skyscrapers made of glass and steel, flying vehicles in the sky, a bustling street market with diverse people, vibrant colors, hyper-realistic details, cinematic lighting, intricate reflections on the buildings, soft pink and orange hues in the sky, ultra-high definition, photorealistic, realistic photo."
negative = "blurry, low quality, pixelated, out of focus, overexposed, underexposed, cartoonish, 2D art, poorly detailed, text artifacts, watermark, deformed buildings, unrealistic proportions, grainy textures, flat colors, monochrome, color bleeding, low resolution, bad composition."

num_inference_steps=50
guidance_scale=7.5

height = 1024  # Altezza in pixel
width = 1024  # Larghezza in pixel

# random seed
seed = random.randint(8, 2**32-1)
print("Using Seed: ",seed)
generator = torch.Generator(device="cpu").manual_seed(seed)

# Generazione dell'immagine
image = pipeline(
    prompt=prompt,
    negative_prompt=negative,                   # Prompt negativo
    height=height,
    width=width,
    num_inference_steps=num_inference_steps,    # Passi di inferenza (maggiore = più dettagli)
    guidance_scale= guidance_scale,              # Peso per il prompt (maggiore = risultati più aderenti)
    generator=generator
).images[0]

# Salva l'immagine generata
image.save("output_image.png")


In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import torch

# Carica la pipeline
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id, 
    torch_dtype=torch.float16
)

# Carica l'immagine di input
input_image = Image.open("00045.png").convert("RGB")  # Assicurati che sia RGB

# Prompt generico (può essere neutro)
prompt = ""

# Applica il processo img2img con strength=0
output = pipeline(
    prompt=prompt, 
    image=input_image, 
    strength=0.0,  # Non introduce noise aggiuntivo
    use_safetensors=True  # Per performance migliori
)

# Salva l'immagine risultante
output_image = output.images[0]
output_image.save("laundered_image.png")

/home/thomasl/anaconda3/envs/launderingTest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  3.65it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingfac